In [30]:
import keras
from keras import layers
import gymnasium as gym
from gymnasium.wrappers.frame_stack import FrameStack
from gymnasium.wrappers.atari_preprocessing import AtariPreprocessing
import numpy as np
import tensorflow as tf
import ale_py


In [31]:
import logging

In [ ]:
from gymnasium.wrappers import RecordEpisodeStatistics

In [34]:

gym.register_envs(ale_py)


In [35]:
env = gym.make('SpaceInvadersNoFrameskip-v4', render_mode="rgb_array")

In [ ]:

seed = 42
gamma = 0.99 # discount_factor
epsilon = 1.0 
epsilon_min = 0.1 
epsilon_max = 1.0
epsilon_interval = (epsilon_max - epsilon_min)
batch_size = 32
max_steps_per_episode = 10000 
max_episodes = 0 # 0 is infinite
max_frames = 1e7 

env = AtariPreprocessing(env)

env = FrameStack(env, 4)
trigger = lambda t: t % 50 == 0
env = gym.wrappers.RecordVideo(env, video_folder="./Outputs/videos", name_prefix='training', episode_trigger=trigger, disable_logger=False)
env = RecordEpisodeStatistics(env)

num_actions = 4

In [ ]:
def create_q_model():
    return keras.Sequential(
        [
            layers.Lambda
            (
                lambda tensor: keras.ops.transpose(tensor, [0, 2, 3, 1]),
                output_shape=(84, 84, 4),
                input_shape=(4, 84, 84)
            ),
            layers.Conv2D(32, kernel_size=8, strides=4, activation="leaky_relu"),
            layers.Conv2D(64, kernel_size=4, strides=2, activation="leaky_relu"),
            layers.Conv2D(64, kernel_size=3, strides=1, activation="leaky_relu"),
            layers.Flatten(),
            layers.Dense(512, activation="leaky_relu"),
            layers.Dense(num_actions, activation="linear")
        ]
    )


model = create_q_model()
model_target = create_q_model()

optimizer = keras.optimizers.Adam(learning_rate=0.00025, clipnorm=1.0)

# observation, _ = env.reset(seed=42)
# state = np.array(observation)
# state_tensor = keras.ops.convert_to_tensor(state)
# state_tensor = keras.ops.expand_dims(state_tensor, 0)
# print(state_tensor)

action_history = []
state_history = []
state_next_history = []
rewards_history = []
done_history = []
episode_reward_history = []
running_reward = 0
episode_count = 0
frame_count = 0

# Number of frames to take random action and observe output
epsilon_random_frames = 50000
# Number of frames for exploration
epsilon_greedy_frames = 1000000.0
# Maximum replay length
# Note: The Deepmind paper suggests 1000000 however this causes memory issues
max_memory_length = 1000000
# Train the model after 4 actions
update_after_actions = 4
# How often to update the target network
update_target_network = 10000
# Using huber loss for stability
loss_function = keras.losses.Huber()


In [ ]:

while True:
    observation, info = env.reset()
    state = np.array(observation)
    episode_reward = 0

    for timestep in range(1, max_steps_per_episode):
        frame_count += 1

        # Use epsilon-greedy for exploration
        if frame_count < epsilon_random_frames or epsilon > np.random.rand(1)[0]:
            # Take random action
            action = np.random.choice(num_actions)
        else:
            # Predict action Q-values
            # From environment state
            state_tensor = keras.ops.convert_to_tensor(state)
            state_tensor = keras.ops.expand_dims(state_tensor, 0)
            action_probs = model(state_tensor, training=False) 
            # Take best action
            action = keras.ops.argmax(action_probs[0]).numpy()
        # Decay probability of taking random action
        epsilon -= epsilon_interval / epsilon_greedy_frames
        epsilon = max(epsilon, epsilon_min)

        # Apply the sampled action in our environment
        state_next, reward, done, _, info = env.step(action)
        state_next = np.array(state_next)

        episode_reward += reward

        # Save actions and states in replay buffer
        action_history.append(action)
        state_history.append(state)
        state_next_history.append(state_next)
        done_history.append(done)
        rewards_history.append(reward)
        state = state_next

        # Update every fourth frame and once batch size is over 32
        if frame_count % update_after_actions == 0 and len(done_history) > batch_size:
            # Get indices of samples for replay buffers
            indices = np.random.choice(
                range(len(done_history)), size=batch_size)

            # Using list comprehension to sample from replay buffer
            state_sample = np.array([state_history[i] for i in indices])
            state_next_sample = np.array(
                [state_next_history[i] for i in indices])
            rewards_sample = [rewards_history[i] for i in indices]
            action_sample = [action_history[i] for i in indices]
            done_sample = keras.ops.convert_to_tensor(
                [float(done_history[i]) for i in indices]
            )

            # Build the updated Q-values for the sampled future states
            # Use the target model for stability
            future_rewards = model_target.predict(state_next_sample, verbose=0)
            # Q value = reward + discount factor * expected future reward
            updated_q_values = rewards_sample + gamma * keras.ops.amax(
                future_rewards, axis=1
            )

            # If final frame set the last value to -1
            updated_q_values = updated_q_values * \
                (1 - done_sample) - done_sample

            # Create a mask so we only calculate loss on the updated Q-values
            masks = keras.ops.one_hot(action_sample, num_actions)

            with tf.GradientTape() as tape:
                # Train the model on the states and updated Q-values
                q_values = model(state_sample)

                # Apply the masks to the Q-values to get the Q-value for action taken
                q_action = keras.ops.sum(
                    keras.ops.multiply(q_values, masks), axis=1)
                # Calculate loss between new Q-value and old Q-value
                loss = loss_function(updated_q_values, q_action)

            # Backpropagation
            grads = tape.gradient(loss, model.trainable_variables)
            optimizer.apply_gradients(zip(grads, model.trainable_variables))

        if frame_count % update_target_network == 0:
            # update the the target network with new weights
            model_target.set_weights(model.get_weights())
            # Log details
            print(f"best score of last 100: {np.max(episode_reward_history)}, running reward: {running_reward:.2f} at episode {episode_count}, frame count {frame_count}")
            model.save(f"./Outputs/breakout_qmodel_{episode_count}.keras")

        # Limit the state and reward history
        if len(rewards_history) > max_memory_length:
            del rewards_history[:1]
            del state_history[:1]
            del state_next_history[:1]
            del action_history[:1]
            del done_history[:1]

        if done:
            break

    # Update running reward to check condition for solving
    episode_reward_history.append(episode_reward)
    if len(episode_reward_history) > 100:
        del episode_reward_history[:1]
    running_reward = np.mean(episode_reward_history)

    logging.info(f"Episode {episode_count}, info['episode']")

    episode_count += 1

    if running_reward > 9000:  # Condition to consider the task solved
        print("Solved at episode {}!".format(episode_count))
        break

    if (
        max_episodes > 0 and episode_count >= max_episodes
    ):  # Maximum number of episodes reached
        print("Stopped at episode {}!".format(episode_count))
        break
    if (max_frames <= frame_count):
        print(f"Stopped at frame {frame_count}!")

MoviePy - Building video d:\workdir\DeepQLab\Outputs\videos\training-episode-0.mp4.
MoviePy - Writing video d:\workdir\DeepQLab\Outputs\videos\training-episode-0.mp4



MoviePy - Done !
MoviePy - video ready d:\workdir\DeepQLab\Outputs\videos\training-episode-0.mp4
best score of last 100: 380.0, running reward: 130.50 at episode 20, frame count 10000
best score of last 100: 440.0, running reward: 115.12 at episode 41, frame count 20000
MoviePy - Building video d:\workdir\DeepQLab\Outputs\videos\training-episode-50.mp4.
MoviePy - Writing video d:\workdir\DeepQLab\Outputs\videos\training-episode-50.mp4



MoviePy - Done !
MoviePy - video ready d:\workdir\DeepQLab\Outputs\videos\training-episode-50.mp4


best score of last 100: 440.0, running reward: 123.58 at episode 60, frame count 30000
best score of last 100: 440.0, running reward: 117.32 at episode 82, frame count 40000
best score of last 100: 440.0, running reward: 122.55 at episode 100, frame count 50000
MoviePy - Building video d:\workdir\DeepQLab\Outputs\videos\training-episode-100.mp4.
MoviePy - Writing video d:\workdir\DeepQLab\Outputs\videos\training-episode-100.mp4



MoviePy - Done !
MoviePy - video ready d:\workdir\DeepQLab\Outputs\videos\training-episode-100.mp4
best score of last 100: 440.0, running reward: 121.45 at episode 118, frame count 60000
best score of last 100: 440.0, running reward: 132.65 at episode 135, frame count 70000
MoviePy - Building video d:\workdir\DeepQLab\Outputs\videos\training-episode-150.mp4.
MoviePy - Writing video d:\workdir\DeepQLab\Outputs\videos\training-episode-150.mp4



...

MoviePy - Done !
MoviePy - video ready d:\workdir\DeepQLab\Outputs\videos\training-episode-400.mp4
best score of last 100: 515.0, running reward: 169.45 at episode 409, frame count 220000
best score of last 100: 455.0, running reward: 159.40 at episode 428, frame count 230000
best score of last 100: 455.0, running reward: 157.10 at episode 446, frame count 240000
MoviePy - Building video d:\workdir\DeepQLab\Outputs\videos\training-episode-450.mp4.
MoviePy - Writing video d:\workdir\DeepQLab\Outputs\videos\training-episode-450.mp4



MoviePy - Done !
MoviePy - video ready d:\workdir\DeepQLab\Outputs\videos\training-episode-450.mp4
best score of last 100: 455.0, running reward: 164.00 at episode 464, frame count 250000
best score of last 100: 715.0, running reward: 165.40 at episode 480, frame count 260000
best score of last 100: 715.0, running reward: 175.35 at episode 497, frame count 270000
MoviePy - Building video d:\workdir\DeepQLab\Outputs\videos\training-episode-500.mp4.
MoviePy - Writing video d:\workdir\DeepQLab\Outputs\videos\training-episode-500.mp4



MoviePy - Done !
MoviePy - video ready d:\workdir\DeepQLab\Outputs\videos\training-episode-500.mp4
best score of last 100: 715.0, running reward: 188.35 at episode 512, frame count 280000
best score of last 100: 715.0, running reward: 190.05 at episode 531, frame count 290000
best score of last 100: 715.0, running reward: 186.30 at episode 549, frame count 300000
MoviePy - Building video d:\workdir\DeepQLab\Outputs\videos\training-episode-550.mp4.
MoviePy - Writing video d:\workdir\DeepQLab\Outputs\videos\training-episode-550.mp4



MoviePy - Done !
MoviePy - video ready d:\workdir\DeepQLab\Outputs\videos\training-episode-550.mp4
best score of last 100: 715.0, running reward: 196.60 at episode 567, frame count 310000
best score of last 100: 685.0, running reward: 196.70 at episode 584, frame count 320000
best score of last 100: 685.0, running reward: 197.05 at episode 599, frame count 330000
MoviePy - Building video d:\workdir\DeepQLab\Outputs\videos\training-episode-600.mp4.
MoviePy - Writing video d:\workdir\DeepQLab\Outputs\videos\training-episode-600.mp4



MoviePy - Done !
MoviePy - video ready d:\workdir\DeepQLab\Outputs\videos\training-episode-600.mp4
best score of last 100: 685.0, running reward: 198.80 at episode 615, frame count 340000
